* [Spojování dat](#Spojování-dat),
    - [spojování s concat](#Spojování-s-concat),
    - [spojování s concat a arg. join](#concat-a-volitelný-argument-join),
    - [spojování s metodou append](#Spojování-pomocí-metody-append),
    - [spojování s merge](#Spojování-pomocí-merge),
    - [spojování s metodou join](#Spojení-pomocí-indexů,-join),
    - [cvičení 1]().
* [Agregace](),
    - [jednoduchá agregace](),
    - [seskupování groupby](),
    - [agregace](),
    - [filtrování](),
    - [transformace](),
    - [apply](),
    - [cvičení 2]().
* [Pivot tabulka](),
    - [úvodní motivace](),
    - [syntaxe tabulky](),
    - [doplňující možnosti](),
    - [cvičení 3]().
* [Časové řady](),
    - [úvodní motivace](),
    - [datum & čas](),
    - [četnosti](),
    - [resampling](),
    - [cvičení 4]().
* [High performance](),
    - [úvodní motivace](),
    - [eval](),
    - [query](),
    - [caveats]().

<br>

## Spojování dat

---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.APOjuHGvkM0wQaUx9ELKAgHaHa%26pid%3DApi&f=1&ipt=a8b2b692c40e190d4b28c164bf13e6c944702a69944035a4ef9053aa6ea1b190&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

Takovými operacemi si můžeš představit jednoduché operace jako **spojování** (*konkatenace*) **dvou a více datasetů** až po složitější joiny podobně jako u databází.

`pandas` obsahuje různé *funkce* a *metody*, které umožňují tento způsob práce.

### Spojování s `concat`

---

Funkce concat() v pandas umožňuje spojit dva nebo více:
1. sloupečků,
2. tabulek.

*Spojení* můžeš provést:
1. **horizontálně**, po sloupcích,
2. **vertikálně**, po řádcích.

Použití funkce `concat()` spočívá v předání seznamu objektů, které chceš spojit, a parametru `axis`, který určuje osu, podle které se májí objekty spojit:

#### Spojení sloupečků

In [ ]:
from pandas import concat, Series, DataFrame

In [ ]:
sloupec_1 = Series(['A', 'B', 'C'], index=[1, 2, 3])
sloupec_2 = Series(['D', 'E', 'F'], index=[4, 5, 6])

In [ ]:
spojene_sloupce = concat([sloupec_1, sloupec_2])

In [ ]:
spojene_sloupce

#### Spojení tabulek

In [ ]:
uzivatele_1 = {
    'jmeno': ['Matouš', 'Marek', 'Lukáš'],
    'vek': [25, 30, 35]
}

In [ ]:
uzivatele_2 = {
    'jmeno': ['Petr', 'Jan', 'Michal'],
    'vek': [40, 45, 50]
}

In [ ]:
df_uzivatele_1 = DataFrame(uzivatele_1)
df_uzivatele_2 = DataFrame(uzivatele_2)

In [ ]:
df_spojene = concat([df_uzivatele_1, df_uzivatele_2], axis=0)

In [ ]:
df_spojene.head()

Tady se ovšem zduplikovali hodnoty některých indexů.

Ty je potřeba opravit tímto postupem:
1. **Vytvořím nový sloupeček** pro indexy,
2. **odstraním starý sloupeček** s duplicitami.

In [ ]:
df_spojene = df_spojene.reset_index()

In [ ]:
df_spojene.head()

In [ ]:
df_spojene = df_spojene.drop("index", axis=1)

In [ ]:
df_spojene

Pro odchytávání duplicitních indexů můžeš doplnit parametr `verify_integrity=True`, případně pokud je irelevantní, ignorovat jej úplně `ignore_index=True`.

### `concat` a volitelný argument `join`

---

V jednoduchých ukázkách, jako jsou ty výše, stačilo tabulky a sloupečky spojit.

To prakticky není vždy ideální řešení, protože některé sloupečky můžou, ale nemusí být shodné.

In [ ]:
from pandas import DataFrame, concat

In [ ]:
df_vzorek_1 = DataFrame({"A": ["A1", "A2"], "B": ["B1", "B2"], "C": ["C1", "C2"]}, index=[1, 2])

In [ ]:
df_vzorek_2 = DataFrame({"B": ["B3", "B4"], "C": ["C3", "C4"], "D": ["D3", "D4"]}, index=[3, 4])

In [ ]:
vystup = concat([df_vzorek_1, df_vzorek_2])

In [ ]:
vystup

Pokud některá data chybějí, jsou automaticky vyplněná jako **neznámé hodnoty**.

Řešením takové situace můžeš být zavedení jiného způsobu spojování.

Tedy nepoužívat defaultní argument `join='outer'`, ale `join='inner'`:

In [ ]:
vystup_bez_na = concat([df_vzorek_1, df_vzorek_2], join='inner')

In [ ]:
vystup_bez_na

### Spojování pomocí metody `append`

---

Jde **o zastaralé řešení**, ale přesto se s ním můžeš setkat.

Jelikož je spojování natolik běžnou operací, vznikl ještě jeden způsob, který je dokonce stručnější jako `concat`.

Jde o metodu `append`:

In [ ]:
df_vzorek_3 = DataFrame({"A": ["A1", "A2"], "B": ["B1", "B2"]}, index=[1, 2])

In [ ]:
df_vzorek_4 = DataFrame({"A": ["A3", "A4"], "B": ["B3", "B4"]}, index=[3, 4])

In [ ]:
df_vzorek_3.append(df_vzorek_4)

Tato metoda přitom neupravovala původní objekty (jako `append` a `extend` pro `list`), ale vytvořila nový objekt.

### Spojování pomocí `merge`

---

Další funkcí pro spojování `DataFrame` objektů je `merge`.

Tato funkce je vhodná pro spojování DataFrame objektů, které **mají společné sloupce**.

Můžeš lépe zadávat typ spojení (parametr `how='inner' | 'outer' | 'left' | 'join'`).

Dále ti umožní definovat sloupec, nebo sloupce, na kterých chceš spojení provést (parametr `on`).

In [ ]:
from pandas import merge

In [ ]:
uzivatele_1 = {
    'jmeno': ['Alice', 'Bob', 'Charlie', 'David'],
     'vek': [25, 30, 35, 40],
     'mesto': ['Brno', 'Praha', 'Plzen', 'Ostrava']
}

In [ ]:
uzivatele_2 = {
    'jmeno': ['Alice', 'David', 'Emma', 'Frank'],
     'pocet_prijemcu': [100, 200, 150, 250]
}

In [ ]:
df_uzivatele_1 = DataFrame(uzivatele_1)

In [ ]:
df_uzivatele_2 = DataFrame(uzivatele_2)

In [ ]:
vystup = merge(df_uzivatele_1, df_uzivatele_2, on='jmeno', how='outer')

In [ ]:
vystup

Obecně platí, že pokud potřebuješ dva nebo více `DataFrame` objektů podle **společného sloupce nebo sloupců**, použij funkci `merge()`.

Pokud chceš jenom **přidat další řádky nebo sloupce** do existujícího `DataFrame` objektu, použij funkci `concat`.

In [ ]:
df_uzivatele_1

In [ ]:
df_uzivatele_2

In [ ]:
vystup_left_join = merge(df_uzivatele_1, df_uzivatele_2, on='jmeno', how='left')

In [ ]:
vystup_left_join

V ukázce výš je použitý *left join*.

Tedy ve výsledku uvidíš celou první (levou tabulku) a z druhé pouze ty záznamy, které mají ve spojovacím sloupci `jmeno` společnou hodnotu.

In [ ]:
df8 = DataFrame({
    'jmeno': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'poradi': [1, 2, 3, 4]
})

In [ ]:
df9 = DataFrame({
    'jmeno': ['Bob', 'Jake', 'Lisa', 'Sue'],
    'poradi': [3, 1, 4, 2]
})

In [ ]:
vystup_konflikt = merge(df8, df9, on="jmeno")

In [ ]:
vystup_konflikt

Funkce sama doplní přípony, aby rozlišila mezi oběma původními sloupci.

Pokud potřebuješ vlastní přípony, můžeš vyzkoušet volitelný argument pro `suffixes`:

In [ ]:
vystup_vlastni_pripony = merge(df8, df9, on='jmeno', suffixes=('_levy', '_pravy'))

In [ ]:
vystup_vlastni_pripony

### Spojení pomocí indexů, `join`

---

Tato metoda slouží k propojení dvou DataFrame objektů na základě jejich indexů nebo hodnot.

Je velice podobná funkci `merge` ale je přímo součástí `DataFrame` objektu a je snazší ji aplikovat:

In [ ]:
uzivatele_1 = {
    'jmeno': ['Alice', 'Bob', 'Petr'],
     'vek': [25, 30, 35]
}

In [ ]:
uzivatele_2 = {
    'jmeno': ['Alice', 'Bob', 'Petr'],
    'pocet_prijemcu': [100, 150, 200]
}

In [ ]:
df_uzivatele_1 = DataFrame(uzivatele_1)

In [ ]:
df_uzivatele_2 = DataFrame(uzivatele_2)

In [ ]:
df_uzivatele_1 = df_uzivatele_1.set_index('jmeno')
df_uzivatele_2 = df_uzivatele_2.set_index('jmeno')

In [ ]:
vysledek_join_metody = df_uzivatele_1.join(df_uzivatele_2)

In [ ]:
vysledek_join_metody

Hlavní rozdíl mezi těmito funkcemi je způsob určení sloupce nebo sloupců, podle kterých se má propojení provést.

Metoda `join()` propojuje DataFrame objekty **na základě jejich indexů**, zatímco funkce `merge()` umožňuje propojit `DataFrame` objekty **na základě hodnoty v jednom nebo více sloupcích**.

<br>

**🧠 CVIČENÍ 🧠, procvič si spojování**

Máš dvě tabulky s informacemi o zákaznících tvé firmy.

Tabulky `zakaznici_objednavky` a `zakaznici_info`.

Tvým úkolem je propojit tyto tabulky podle zadání:
1. Použij funkci `merge()` k propojení obou tabulek podle společného klíče, kterým je sloupec `id_zakaznika`,
2. Použij funkci `concat()` k přidání sloupce `celkova_cena_objednavky`, který bude vypočítán jako součet ceny všech objednávek daného zákazníka,
3. Vyfiltruj pouze informace o zákaznících ze státu `'USA'` a ulož výslednou tabulku.

In [ ]:
import pandas as pd

In [ ]:
df_zakaznici_info = pd.DataFrame({'id_zakaznika': [1, 2, 3, 4],
                               'jmeno': ['Jan Novák', 'Petr Soukup', 'Marie Horáková', 'Jana Svobodová'],
                               'adresa': ['Hlavní 15', 'Druhá 10', 'Třetí 25', 'Čtvrtá 20'],
                               'mesto': ['Praha', 'Brno', 'Ostrava', 'New York'],
                               'stat': ['CZ', 'CZ', 'CZ', 'USA']})

In [ ]:
df_zakaznici_objednavky = pd.DataFrame({'id_zakaznika': [1, 2, 3, 4, 1, 2],
                                     'datum_objednavky': ['2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01'],
                                     'nazev_produktu': ['PC', 'Notebook', 'Monitor', 'Tiskarna', 'Myš', 'Klávesnice'],
                                     'cena': [15000, 20000, 5000, 3000, 500, 800]})

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    spojene_tabulky = pd.merge(df_zakaznici_info, df_zakaznici_objednavky, on='id_zakaznika')

    celkova_cena_objednavek = spojene_tabulky.groupby('id_zakaznika')['cena'].sum().reset_index()
    celkova_cena_objednavek = celkova_cena_objednavek.rename(columns={'cena': 'celkova_cena_objednavky'})
    spojene_tabulky = pd.concat([spojene_tabulky, celkova_cena_objednavek['celkova_cena_objednavky']], axis=1)

    spojene_tabulky = spojene_tabulky[spojene_tabulky['stat'] == 'USA']
    ```
</details>

## Agregace

Seskupování a agregace jsou procesy, které patří k základní efektivní analýze dat.

### Jednoduchá agregace

Přesto, že veškeré základní **statistické údaje** prakticky nabízí metoda `describe()`, můžeš ocenit, když stejnou statistiku můžeš aplikovat **na tebou vybrané objekty**.

Mezi nejjednodušší postupy, jak data analyzovat patří metody jako:
* `sum()`,
* `mean()`,
* `median()`,
* `min()`,
* `max()`.

Všechny tyto metody umožní získat jedno samotné číslo, které ti umožní prohlédnout podstatu zadaného datasetu.

In [2]:
from pandas import read_csv

In [4]:
df_nemovitosti = read_csv("nemovitosti.csv")

<br>

#### Odstranit nepotřebný sloupeček

In [11]:
df_nemovitosti.columns

Index(['Unnamed: 0', 'id', 'price', 'area', 'bedrooms', 'bathrooms', 'garage',
       'distance_to_center'],
      dtype='object')

In [14]:
df_bez_bezejmen = df_nemovitosti.drop("Unnamed: 0", axis=1)

In [18]:
df_bez_bezejmen.head()

,id,price,area,bedrooms,bathrooms,garage,distance_to_center
0,1,18094478,254,1,1,False,16
1,2,15315092,228,1,3,False,24
2,3,4234489,112,3,3,False,16
3,4,16586186,145,2,1,True,23
4,5,11628519,280,5,1,False,2


In [17]:
df_bez_bezejmen.loc[:, "price"].max()

19824013

In [ ]:
df_bez_bezejmen.loc[:, "price"].min()

In [ ]:
df_bez_bezejmen.loc[:, "area"].sum()

In [ ]:
df_bez_bezejmen.loc[:, "price"].mean()

<br>

Často ale není dostačující, prozkoumat data pouze jednoduchých agregačních funkcí.

Další operace, které je potřeba pochopit jsou seskupování dat podle zadaných parametrů.

### Seskupování groupby

Funkce `groupby`, původně operace z SQL jazyka, je v rámci knihovny `pandas` všestraný pomocník pro seskupování dat na základě různých kritérií.

In [2]:
from pandas import DataFrame

In [3]:
df_pokus_s_cisly = DataFrame(
    {
        'klíč': ['A', 'B', 'C', 'A', 'B', 'C'],
        'číselná hodnota': range(6)},
)

In [4]:
df_pokus_s_cisly.head(6)

,klíč,číselná hodnota
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


#### Klasické seskupení podle sloupečku

In [5]:
df_pokus_s_cisly.groupby("klíč")

Metoda standardně vrací `DataFrameGroupBy`.

Jde opět o tzv. *lazy evaluation* proces, samotný nic neprovede, pouze čeká na pokyn uživatele, který samotnou agregaci spustí.

In [6]:
df_pokus_s_cisly.groupby("klíč").sum()

,číselná hodnota
klíč,
A,3
B,5
C,7


Metoda `sum()` je pouze jednou z možností, se kterou můžeš pracovat.

### Sloupečkové označování

Stejně jako `DataFrame` můžeš označovat také *GroupBy* objekty.

In [7]:
df_pokus_s_cisly.groupby("klíč")["číselná hodnota"].median()

klíč
A    1.5
B    2.5
C    3.5
Name: číselná hodnota, dtype: float64

<br>

V uplynulé ukázce je zadaná seskupování podle sloupečku `klíč`.

Dále je vybraný pouze konkrétní sloupeček, na který chceš spustit metodu `median`.

<br>

Pokud potřebuješ nad vybraným objektem provádět některé procesy ručně, můžeš přes *GroupBy* objekt **iterovat**:

In [11]:
for (klic, hodnota) in df_pokus_s_cisly.groupby("klíč"):
    print(f"Klic: {klic}; Hodnota={hodnota.shape}")

Klic: A; Hodnota=(2, 2)
Klic: B; Hodnota=(2, 2)
Klic: C; Hodnota=(2, 2)


### Agregace

Kromě jednoduchých agregací, nabízí *GroupBy* řadu další funkcionality.

Jde o metody:
* `aggregate`,
* `filter`,
* `transform`,
* `apply`.

In [24]:
import numpy

In [25]:
rng = numpy.random.RandomState(0)

In [19]:
from pandas import DataFrame

In [26]:
df_pokus_s_cisly = DataFrame(
    {
        'klíč': ['A', 'B', 'C', 'A', 'B', 'C'],
        'data_1': range(6),
        'data_2': rng.randint(0, 10, 6)
    }
)

In [28]:
df_pokus_s_cisly.head()

,klíč,data_1,data_2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7


Statistick metody určitě nabízí spousty pomůcek.

Objekt typu *GroupBy* umí pracovat také s funkcemi, stringy a celými listy.

In [29]:
df_pokus_s_cisly.groupby("klíč").aggregate(["min", numpy.median, max])

data_1            data_2           
        min median max    min median max
klíč                                    
A         0    1.5   3      3    4.0   5
B         1    2.5   4      0    3.5   7
C         2    3.5   5      3    6.0   9

Můžeš říct, že metoda `aggregate` se používá k aplikaci **jedné nebo více agregačních funkcí na seskupená data**.

```
store,fruit,quantity_sold,price
A,apple,10,20
A,banana,15,12
B,apple,8,22
B,banana,30,10
C,apple,20,18
C,banana,25,15
```

In [30]:
ovoce_data = {'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
        'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
        'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
        'cena': [20, 12, 22, 10, 18, 15]}

In [31]:
df_ovoce = DataFrame(ovoce_data)

In [35]:
df_ovoce.head(6)

,pobocka,ovoce,prodane_mnozstvi,cena
0,A,jablko,10,20
1,A,banan,15,12
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


In [36]:
vystup = df_ovoce.groupby('pobocka').aggregate({'prodane_mnozstvi': 'sum', 'cena': 'mean'})

In [37]:
vystup

,prodane_mnozstvi,cena
pobocka,,
A,25,16.0
B,38,16.0
C,45,16.5


1. Nejprve jsou hodnoty seskupení podle sloupce `pobocka`,
2. poté specifikuješ pomocí `aggregate` funkce a sloupce,
3. .. tedy sumarizovat hodnoty v `prodane_mnozstvi` a získat průměr `cena` pro každou pobočku.

### Filtrování

Filtrování ti umožní zahodit takové údaje, které nesplňují zadanou podmínku.

Metoda `filter` se používá k vybrání seskupených dat podle splnění určité podmínky.

Představme si, že máme následující dataset s informacemi o prodeji ovoce v různých obchodech:

In [41]:
def vyber_pouze_zadane_mnozstvi(x, limit: int = 35):
    return x['prodane_mnozstvi'].sum() > limit

<br>

Funkce `vyber_pouze_zadane_mnozstvi`, tedy **filtrovací funkce**, musí vracet **boolean** datový typ.

In [42]:
vysledek = df_ovoce.groupby('pobocka').filter(vyber_pouze_zadane_mnozstvi)

In [43]:
vysledek

,pobocka,ovoce,prodane_mnozstvi,cena
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `filter`, která umožňuje použít uživatelem definovanou funkci,
3. definuješ funkci, která vybere pouze pobočky s větším prodejem než je parametr `limit`,
4. metoda `filter` pak vybere pouze ty řádky, které splňují tuto podmínku.

### Transformace

Zatímco předchozí výsledky *agregace pomocí `groupby` vraceli redukované množství dat.

Transformace obvykle vrací data o stejném rozsahu jako vstupní data. Jenom upravená.

Metoda `transform` se používá k aplikaci určité **transformační funkce** na každý prvek seskupených dat.

In [56]:
def vrat_procenta_z_celkoveho_prodeje(udaj):
    return round(udaj / udaj.sum() * 100, 1)

<br>

Vytvoření nového sloupečku `procento_z_celkoveho_prodeje`:

In [57]:
df_ovoce['procento_z_celkoveho_prodeje'] = df_ovoce.groupby('pobocka')['prodane_mnozstvi'] \
    .transform(vrat_procenta_z_celkoveho_prodeje)

In [58]:
df_ovoce.head(6)

,pobocka,ovoce,prodane_mnozstvi,cena,procento_z_celkoveho_prodeje
0,A,jablko,10,20,40.0
1,A,banan,15,12,60.0
2,B,jablko,8,22,21.1
3,B,banan,30,10,78.9
4,C,jablko,20,18,44.4
5,C,banan,25,15,55.6


1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `transform`, která umožňuje použít uživatelem definovanou funkci,
3. definuješ funkci, která vybere vypočítá procentuální vyjádření prodaného zboží pro pobočku,
4. přidáš nový sloupeček `procento_z_celkoveho_prodeje`.

### Metoda `apply`

Metoda `apply` ti také dovolí, používat uživatelem definované funkce na seskupená data.

Následně vrací objekt knihovny `pandas` (buď `DataFrame`, nebo `Series`, a nebo skalární hodnota).

In [7]:
df_ovoce.head(6)

,pobocka,ovoce,prodane_mnozstvi,cena
0,A,jablko,10,20
1,A,banan,15,12
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


In [44]:
def vypocitej_vydelek_za_artikl(seskupene):
    seskupene['vydelek_pobocky'] = seskupene['prodane_mnozstvi'] * seskupene['cena']
    return seskupene

<br>

Metoda `apply` je **obecnější a flexibilnější** než `transform`.

`apply` umožňuje použít uživatelem definovanou funkci na každou skupinu po seskupení dat pomocí `groupby`.

Výsledek metody `apply` může mít jiný tvar než původní data.

In [48]:
vystup = df_ovoce.groupby('pobocka').apply(vypocitej_vydelek_za_artikl)

/tmp/ipykernel_10497/1874034112.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  vystup = df_ovoce.groupby('pobocka').apply(vypocitej_vydelek_za_artikl)


In [15]:
df_ovoce.groupby?

Signature:
df_ovoce.groupby(
    by=None,
    axis: 'Axis' = 0,
    level: 'IndexLabel | None' = None,
    as_index: 'bool' = True,
    sort: 'bool' = True,
    group_keys: 'bool | lib.NoDefault' = <no_default>,
    squeeze: 'bool | lib.NoDefault' = <no_default>,
    observed: 'bool' = False,
    dropna: 'bool' = True,
) -> 'DataFrameGroupBy'
Docstring:
Group DataFrame using a mapper or by a Series of columns.

A groupby operation involves some combination of splitting the
object, applying a function, and combining the results. This can be
used to group large amounts of data and compute operations on these
groups.

Parameters
----------
by : mapping, function, label, or list of labels
    Used to determine the groups for the groupby.
    If ``by`` is a function, it's called on each value of the object's
    index. If a dict or Series is passed, the Series or dict VALUES
    will be used to determine the groups (the Series' values are first
    aligned; see ``.align()`` method). If a li

Od posledních verzí frameworku (`1.5.0` a vyšší) platí, že pokud bude výsledkem `DataFrame` nebo `Series` musíš uvést argument pro `group_keys=True`).

In [49]:
vystup = df_ovoce.groupby('pobocka', group_keys=True).apply(vypocitej_vydelek_za_artikl)

In [50]:
vystup.head(6)

pobocka   ovoce  prodane_mnozstvi  cena  vydelek_pobocky
pobocka                                                           
A       0       A  jablko                10    20              200
        1       A   banan                15    12              180
B       2       B  jablko                 8    22              176
        3       B   banan                30    10              300
C       4       C  jablko                20    18              360
        5       C   banan                25    15              375

Na první pohled vypadají metody `apply` a `transform` docela podobně.

Metoda `transform` je trochu omezenější než `apply`.

Slouží k aplikaci uživatelem definované nebo vestavěné funkce **na každý prvek** skupiny po seskupení s `groupby`.

`transform` musí vracet hodnotu stejného tvaru jako vstupní data.

Výsledek metody `transform` **má stejný tvar jako původní data**.

#### Apply
* potřebuješ výsledek, který **má jiný tvar než původní data**,
* umí zpracovat **více sloupečků současně**.

#### Transform
* potřebuješ výsledek, který **má stejný tvar jako původní data**,
* umí zpracovat **pouze jeden sloupeček**.

In [70]:
df_rozdily = DataFrame({
    'KLIC': ['A','B','C'] * 3,
    'A': np.arange(9),
    'B': [1,2,3] * 3,
})

In [71]:
df_rozdily

,KLIC,A,B
0,A,0,1
1,B,1,2
2,C,2,3
3,A,3,1
4,B,4,2
5,C,5,3
6,A,6,1
7,B,7,2
8,C,8,3


#### `transform` vrací výsledky ve stejném tvaru

In [72]:
def vypocitej_sumu(data):
    return data.sum()

In [76]:
seskup_df_rozdily_apply = df_rozdily.groupby('KLIC')['A'].apply(vypocitej_sumu)

In [77]:
seskup_df_rozdily_apply

KLIC
A     9
B    12
C    15
Name: A, dtype: int64

In [78]:
seskup_df_rozdily_trans = df_rozdily.groupby('KLIC')['A'].transform(vypocitej_sumu)

In [79]:
seskup_df_rozdily_trans

0     9
1    12
2    15
3     9
4    12
5    15
6     9
7    12
8    15
Name: A, dtype: int64

#### `apply` umí pracovat s více sloupečky, `transform` jen s jedním

In [80]:
def vypocitej_rozdil(data):
    return data['B'] - data['A']

In [81]:
df_rozdily.groupby('KLIC').apply(vypocitej_rozdil)

KLIC   
A     0    1
      3   -2
      6   -5
B     1    1
      4   -2
      7   -5
C     2    1
      5   -2
      8   -5
dtype: int64

In [82]:
df_rozdily.groupby('KLIC').transform(vypocitej_rozdil)

KeyError: 'B'

<br>

**🧠 CVIČENÍ 🧠, procvič si funkcí GroupBy a agregační funkce**

Máš zadaný takový datový set.
```
store,fruit,quantity_sold,price
A,apple,10,20
A,banana,15,12
B,apple,8,22
B,banana,30,10
C,apple,20,18
C,banana,25,15
```

Následně:
1. Pomocí metody `filter` vyber prodejny, které prodaly **alespoň 30 produktů**,
2. na filtrovaném datasetu použijte metodu `apply` pro výpočet **celkového příjmu z prodeje pro každý obchod**.

In [83]:
from pandas import DataFrame

In [100]:
df_prodej_hardware = DataFrame({
    'prodejna_id': (5, 4, 1, 5, 5, 1, 4, 2, 5, 1, 3, 1, 3, 4, 2, 1, 5, 4, 1, 5),
    'transakce_id': (1278, 1216, 1866, 1872, 1797, 1272, 1880, 1061, 1595, 1879, 1728,
       1341, 1396, 1698, 1018, 1176, 1611, 1395, 1444, 1232),
    'predmet_prodeje': ('grafická_karta', 'SSD', 'RAM', 'procesor', 'grafická_karta',
       'základní_deska', 'SSD', 'SSD', 'grafická_karta', 'RAM',
       'grafická_karta', 'procesor', 'grafická_karta', 'SSD',
       'grafická_karta', 'RAM', 'základní_deska', 'HDD', 'grafická_karta',
       'RAM'),
    'pocet_prodanych_ks': (1,  5,  6,  6,  3,  7,  9, 10,  8,  6,  8,  5,  8, 10,  4, 10,  8,
       10,  2,  5),
    'cena_predmetu': (19500.69874949, 19731.10951735, 14114.15342339, 10953.87914371,
        6535.78851758, 16369.00288429, 13852.2578648 ,  3671.03031723,
       18263.08009763, 16539.476237  , 19021.09830919, 14651.53041357,
       12461.59632075,  8655.73920767, 18688.2054254 , 17388.24584526,
        1381.76406707,  1014.1560027 ,  7841.03565412, 16305.78995025)
})

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    def vyber_prodej_vetsi_nez_limit(data, limit: int = 30):
        return data['pocet_prodanych_ks'].sum() > limit
        
    def vypocitej_celkovy_vydelek_prodejny(skupina):
        return (skupina['pocet_prodanych_ks'] * skupina['cena_predmetu']).sum()
    
    df_filtr_hardware = df_prodej_hardware.groupby('prodejna_id') \
                            .filter(vyber_prodej_vetsi_nez_limit)
    
    
    celkovy_vydelek = df_filtr_hardware.groupby('prodejna_id') \
                            .apply(vypocitej_celkovy_vydelek_prodejny)
    ```
</details>

---

## Pivot tabulky

Pivot tabulky jsou užitečné pro přehledné zobrazení a analýzu dat z tabulkových zdrojů.

Pomocí pivot tabulek můžeš seskupit data podle určitých kategorií a provést agregaci hodnot.

Nejprve si představ situaci bez pivot tabulek, pomocí ukázky níže:

In [143]:
!pip install seaborn

     |████████████████████████████████| 293 kB 2.7 MB/s eta 0:00:01


In [144]:
import numpy as np
import pandas as pd
import seaborn as sns

V této ukazce použiješ vzorová data týkající se nehody Titaniku:

In [145]:
titanic = sns.load_dataset('titanic')

In [146]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


<br>

Pro jakoukoliv pokročilou analýzu dat, potřebuješ údaj seskupit.

In [148]:
titanic.groupby('sex')[['survived']].mean().round(2)

,survived
sex,
female,0.74
male,0.19


Takový průzkum z této studie ti dá jasný pohled na věc:
* 3 ze 4 žen přežily,
* 1 z 5 mužů přežil.

Pokud budeš potřebovat detailnější analýzy, budeš potřebovat více dat.

Třeba situaci, kde kromě pohlaví, bereš v potaz **třídu cestujících**:

In [154]:
titanic.groupby(['sex', 'class'])['survived'].mean().round(2)

sex     class 
female  First     0.97
        Second    0.92
        Third     0.50
male    First     0.37
        Second    0.16
        Third     0.14
Name: survived, dtype: float64

<br>

Metodou `unstack` si můžeš vytvořit nové sloupečky, které jsou postavené na novém Indexu, nebo Indexech (*Multiindex*):

In [155]:
titanic.groupby(['sex', 'class'])['survived'].mean().round(2).unstack()

class,First,Second,Third
sex,,,
female,0.97,0.92,0.50
male,0.37,0.16,0.14


Takový průzkum ti dá skutečně lepší pohled na věc.

Současně ale roste náročnost ohlášení. Zápis "bobtná" a stává se náročnějším na přečtení a pochopení. 

### Pivot tabulka

Podobné řešení ti nabízí funkce `pivot_table`:

In [161]:
titanic.pivot_table('survived', index='sex', columns='class').round(2)

class,First,Second,Third
sex,,,
female,0.97,0.92,0.50
male,0.37,0.16,0.14


Zásádním rozdílem je ovšem čitelnost, kterou máš pro tuto variantu zápisu.

Pomocí vhodných argumentů, můžeš doplnit vysvětlivky tam, kde funkce `groupby` nemohla.

Stejně platí, že pokud budeš potřebovat další Index, můžeš si pomoci funkcí `cut`:

In [158]:
age = pd.cut(titanic['age'], [0, 18, 80])

In [160]:
titanic.pivot_table('survived', ['sex', age], 'class').round(2)

class            First  Second  Third
sex    age                           
female (0, 18]    0.91    1.00   0.51
       (18, 80]   0.97    0.90   0.42
male   (0, 18]    0.80    0.60   0.22
       (18, 80]   0.38    0.07   0.13

<br>

**🧠 CVIČENÍ 🧠, procvič si pivot tabulky**

Ze zadaného datasetu vytvoř **pivot tabulku**, která zobrazí **počet prodaných kusů ovoce** pro **každý obchod** a **druh ovoce**.

In [162]:
ovoce_data = {'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
        'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
        'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
        'cena': [20, 12, 22, 10, 18, 15]}

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    pivot_tabulka = df_ovoce.pivot_table(values="prodane_mnozstvi", index="pobocka", columns="ovoce")
    ```
</details>

## Časové řady

* úvodní motivace,
* datum & čas,
* četnosti,
* resampling,
* cvičení 4.

Část frameworku byla vyvinuta za účelem finančního modelování.

Proto je více než dobře vybavena sadou nástrojů, které umí pracovat s daty, časem a časovými objekty.

Jde například o údaje typu:
* *timestampy*, údaj odkazující na konkrétní časový okamžik (např. 4. července 2015 v 7:00 hod.),
* *časové intervaly*, tedy období odkazují na délku času mezi konkrétním začátkem a koncem (např. intervaly ze dne na den),
* *time delta* objekty, tedy přesné délky času (např. 22,22 sekundy).

### Data a čas v Pythonu

Standardní výbavou Pythonu jsou knihovny `datetime` a `dateutil`:

In [167]:
from datetime import datetime

In [168]:
datetime(year=2023, month=4, day=5)

datetime.datetime(2023, 4, 5, 0, 0)

<br>

Nebo knihovna pro parsování datových typů z různých stringových zadání:

In [169]:
from dateutil import parser

In [172]:
date = parser.parse("5th of april, 2023")

In [173]:
date

datetime.datetime(2023, 4, 5, 0, 0)

Kde pomocí metody `strftime` můžeš vypsat den:

In [174]:
date.strftime("%A")

'Wednesday'

### Data a čas v numpy

Některé nedostatky uvnitř knihoven `datetime` a `dateutil` vedli ke vzniku sady nástrojů.

Tyto doplňky vznikly pod hlavičkou knihovny `numpy`.

In [180]:
from numpy import array, arange

In [178]:
date = array('2023-04-05', dtype=np.datetime64)

In [179]:
date

array('2023-04-05', dtype='datetime64[D]')

<br>

Pokud potřebuješ pole následujících 7 dní:

In [181]:
date + arange(7)

array(['2023-04-05', '2023-04-06', '2023-04-07', '2023-04-08',
       '2023-04-09', '2023-04-10', '2023-04-11'], dtype='datetime64[D]')

Vzhledem k jednotnému datovu typu v poli pro **numpy** `datetime64` může tento typ operace
provádět mnohem rychleji, než přímo v Pythonu `datetime` objekty, zejména když objekty nabývají na velikosti.

### Data a čas v pandách

Jde o kombinace objektů z obou předchozích podkapitol.

Ty dávají dohromady to nejlepší prostředky pro zacházení s časem.

In [184]:
from pandas import to_datetime

In [185]:
date = to_datetime("5th of April, 2023")

In [186]:
date

Timestamp('2023-04-05 00:00:00')

In [187]:
date.strftime("%A")

'Wednesday'

### Časové řady

V podstatě jde o hlavní nástroj, který tato knihovna dovede nabídnout.

#### Indexovní časem

Ukázka, kde vytvoříš sloupeček, který obsahuje jenom *dummy* **data a datumy**:

In [193]:
from pandas import DatetimeIndex, Series

In [194]:
datumy = ["2023-04-05", "2022-04-05", "2021-04-05", "2020-04-05"]

In [195]:
indexy = DatetimeIndex(datumy)

In [196]:
hodnoty = [to_datetime(den).strftime("%A") for den in datumy]

In [197]:
df_hodnoty = Series(hodnoty, index=indexy)

In [198]:
df_hodnoty

2023-04-05    Wednesday
2022-04-05      Tuesday
2021-04-05       Monday
2020-04-05       Sunday
dtype: object

In [199]:
type(indexy)

pandas.core.indexes.datetimes.DatetimeIndex

In [202]:
df_hodnoty["2020": "2022"]

/tmp/ipykernel_10497/3590313258.py:1: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  df_hodnoty["2020": "2022"]


2022-04-05    Tuesday
2021-04-05     Monday
2020-04-05     Sunday
dtype: object

In [ ]:
#### Základní objekty

In [ ]:
Timestamp + DatetimeIndex

In [ ]:
date_range

In [ ]:
frequencies and offsets

In [ ]:
resampling, shifting, windwing

---